In [ ]:
import cv2
import os
import numpy as np

# Path to the directory containing training images
img_dir = 'path_to_your_training_images'

# Initialize empty lists for storing faces and corresponding labels
faces = []
ids = []

# Step 1: Train FaceRecognizer model

def detect_faces(image):
    """Detect faces in the image and return the face regions."""
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    faces_detected = face_cascade.detectMultiScale(gray_img, 1.3, 5)  # Detect faces
    face_regions = []
    
    for (x, y, w, h) in faces_detected:
        face_region = gray_img[y:y+h, x:x+w]  # Extract face region
        face_regions.append((face_region, (x, y, w, h)))  # Append both the face and bounding box
    return face_regions


# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Loop through each folder (representing different people) in the image directory
for face_id in os.listdir(img_dir):
    face_path = os.path.join(img_dir, face_id)
    
    if os.path.isdir(face_path):  # Check if it's a valid directory (person folder)
        # Loop through each image in the person folder
        for img_label in os.listdir(face_path):
            img_path = os.path.join(face_path, img_label)
            img = cv2.imread(img_path)
            
            # Detect faces in the image using the reusable function
            face_regions = detect_faces(img)

            # Add the detected faces and their labels to the training lists
            for face_region, _ in face_regions:
                faces.append(face_region)  # Add the face to the faces list
                ids.append(int(face_id))  # Add the label (person's ID) to the labels list

# Step 2: Train the recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()  # Initialize the face recognizer
recognizer.train(faces, np.array(ids))  # Train the recognizer with the faces and their labels

# Step 3: Save the trained model
recognizer.save('trained_model.yml')  # Save the trained model for future use
print("Training complete and model saved as 'trained_model.yml'.")

# --------------------------------------------

# Step 4: Load the trained model for face recognition
recognizer.read('trained_model.yml')

# Path to the image(s) you want to recognize (you can replace this with real-time video capture)
test_image_path = 'path_to_test_image.jpg'

# Load the test image
test_img = cv2.imread(test_image_path)

# Detect faces in the test image using the same reusable function
face_regions = detect_faces(test_img)

# Step 5: Recognize faces in the test image
for face_region, (x, y, w, h) in face_regions:
    # Recognize the face using the trained recognizer
    label, confidence = recognizer.predict(face_region)
    
    # Draw a rectangle around the face and display the label
    color = (0, 255, 0)  # Green color for recognized faces
    if confidence < 100:  # If the confidence is low (indicating a good match)
        cv2.rectangle(test_img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(test_img, f'Person {label}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    else:
        # If the confidence is high, mark the face as unknown
        cv2.rectangle(test_img, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red color for unknown faces
        cv2.putText(test_img, 'Unknown', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

# Step 6: Show the result
cv2.imshow("Recognized Faces", test_img)
cv2.waitKey(0)  # Wait for any key to close the window
cv2.destroyAllWindows()
